# Gestor de investimento
---

In [1]:
import pandas as pd


# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [31]:
df = pd.read_excel('../PosicaoHistorico_200091-2.xls')

WARNING *** file size (71838) not 512 + multiple of sector size (512)


In [44]:
df[~df['Unnamed: 2'].isnull()].head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84
2,nan,NaN,Papel,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
6,nan,NaN,BPAC11,nan,nan,nan,nan,nan,nan,nan,200,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,nan,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,nan,200,nan,NaN,nan,nan,nan,NaN,nan,42.30,NaN,nan,nan,nan,nan,NaN,"8,460.00",NaN
8,nan,NaN,JHSF3,nan,nan,nan,nan,nan,nan,nan,1600,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,nan,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,nan,1600,nan,NaN,nan,nan,nan,NaN,nan,4.16,NaN,nan,nan,nan,nan,NaN,"6,656.00",NaN
10,nan,NaN,MGLU3,nan,nan,nan,nan,nan,nan,nan,800,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,nan,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,nan,800,nan,NaN,nan,nan,nan,NaN,nan,49.70,NaN,nan,nan,nan,nan,NaN,"39,760.00",NaN
12,nan,NaN,PETR4,nan,nan,nan,nan,nan,nan,nan,400,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,nan,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,nan,400,nan,NaN,nan,nan,nan,NaN,nan,18.05,NaN,nan,nan,nan,nan,NaN,"7,220.00",NaN
14,nan,NaN,WEGE3,nan,nan,nan,nan,nan,nan,nan,300,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,nan,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,nan,300,nan,NaN,nan,nan,nan,NaN,nan,39.94,NaN,nan,nan,nan,nan,NaN,"11,982.00",NaN
17,nan,NaN,Opções,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,nan,0,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
19,nan,NaN,Termos,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,nan,0,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
30,nan,NaN,Aluguel (BTC),nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,nan,nan,

In [84]:
# Função para pegar os dados de Papel (ações)

stocks = []

for index, row in df[~df['Unnamed: 2'].isnull()].iterrows():
    if row['Unnamed: 2'] == 'Papel':
        continue
    elif row['Unnamed: 2'] == 'Opções':
        break
    else:
        # continue
        print(row[['Unnamed: 2', 'Unnamed: 10', 'Unnamed: 18']])
        stocks.append(row[['Unnamed: 2', 'Unnamed: 10', 'Unnamed: 18']])

pd_stock = pd.DataFrame(columns=['Papel', 'Qtd. Disponivel', 'Qtde. Projetado'], data=stocks)

pd_stock.head()

Unnamed: 2     BPAC11
Unnamed: 10       200
Unnamed: 18         0
Name: 6, dtype: object
Unnamed: 2     JHSF3
Unnamed: 10     1600
Unnamed: 18        0
Name: 8, dtype: object
Unnamed: 2     MGLU3
Unnamed: 10      800
Unnamed: 18        0
Name: 10, dtype: object
Unnamed: 2     PETR4
Unnamed: 10      400
Unnamed: 18        0
Name: 12, dtype: object
Unnamed: 2     WEGE3
Unnamed: 10      300
Unnamed: 18        0
Name: 14, dtype: object


,Papel,Qtd. Disponivel,Qtde. Projetado
6,nan,nan,nan
8,nan,nan,nan
10,nan,nan,nan
12,nan,nan,nan
14,nan,nan,nan


In [48]:
df[~df['Unnamed: 2'].isnull()]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84
2,nan,NaN,Papel,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,nan,NaN,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
6,nan,NaN,BPAC11,nan,nan,nan,nan,nan,nan,nan,200,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,nan,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,nan,200,nan,NaN,nan,nan,nan,NaN,nan,42.30,NaN,nan,nan,nan,nan,NaN,"8,460.00",NaN
8,nan,NaN,JHSF3,nan,nan,nan,nan,nan,nan,nan,1600,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,nan,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,nan,1600,nan,NaN,nan,nan,nan,NaN,nan,4.16,NaN,nan,nan,nan,nan,NaN,"6,656.00",NaN
10,nan,NaN,MGLU3,nan,nan,nan,nan,nan,nan,nan,800,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,nan,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,nan,800,nan,NaN,nan,nan,nan,NaN,nan,49.70,NaN,nan,nan,nan,nan,NaN,"39,760.00",NaN
12,nan,NaN,PETR4,nan,nan,nan,nan,nan,nan,nan,400,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,nan,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,nan,400,nan,NaN,nan,nan,nan,NaN,nan,18.05,NaN,nan,nan,nan,nan,NaN,"7,220.00",NaN
14,nan,NaN,WEGE3,nan,nan,nan,nan,nan,nan,nan,300,NaN,nan,NaT,NaN,nan,nan,nan,0,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,0,nan,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,NaN,nan,nan,0,nan,nan,NaN,nan,NaN,nan,NaN,0,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,0,nan,nan,NaN,nan,nan,nan,300,nan,NaN,nan,nan,nan,NaN,nan,39.94,NaN,nan,nan,nan,nan,NaN,"11,982.00",NaN
17,nan,NaN,Opções,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,nan,0,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
19,nan,NaN,Termos,nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,nan,0,nan,NaN,nan,nan,nan,NaN,nan,NaN,NaN,nan,nan,nan,nan,NaN,nan,NaN
30,nan,NaN,Aluguel (BTC),nan,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,NaT,NaN,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,nan,NaN,NaN,nan,NaN,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,NaN,nan,NaN,nan,nan,NaN,nan,nan,NaN,nan,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,nan,nan,